In [2]:
import numpy as np
import pandas as pd


from sklearn.cross_validation import cross_val_score

import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer,TfidfTransformer
from sklearn import pipeline, preprocessing, feature_extraction
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import os
from sklearn.metrics import accuracy_score

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
d = 'data/pan18-style-change-detection-training-dataset-2018-01-31/'
train_files = os.listdir(d)

truth_files = []
text_files = []
for i in train_files:
    if 'truth' in i:
        truth_files += [i]
    else:
        text_files += [i]
        
truth_files = np.sort(truth_files)
text_files = np.sort(text_files)


texts = []
truths = []
for i, j in zip(text_files, truth_files):
    file = open(d+i, 'r') 
    texts += [file.read()]
    
    file = open(d+j, 'r') 
    truths += [file.read()]
    
df = pd.DataFrame(texts)
df.columns = ['text']
df['coord'] = truths
df['coord'] = df['coord'].apply(lambda x: x.split('[')[1].split(']')[0])


d = 'data/pan18-style-change-detection-validation-dataset-2018-01-31/'
train_files = os.listdir(d)

truth_files = []
text_files = []
for i in train_files:
    if 'truth' in i:
        truth_files += [i]
    else:
        text_files += [i]
        
truth_files = np.sort(truth_files)
text_files = np.sort(text_files)


texts = []
truths = []
for i, j in zip(text_files, truth_files):
    file = open(d+i, 'r') 
    texts += [file.read()]
    
    file = open(d+j, 'r') 
    truths += [file.read()]
    
df_val = pd.DataFrame(texts)
df_val.columns = ['text']
df_val['coord'] = truths
df_val['coord'] = df_val['coord'].apply(lambda x: x.split('[')[1].split(']')[0])


df = pd.concat([df, df_val], axis=0)
df.index = range(len(df))
y = df['coord'].apply(lambda x: len(x) > 0)

In [4]:
skf = StratifiedKFold(5, shuffle=True, random_state=1)
folds = []
fold_id = 0
df['fold_id'] = np.nan
for itr, ite in skf.split(df, y):
    folds += [[itr, ite]]
    df.ix[ite, 'fold_id'] = fold_id
    fold_id += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [16]:
bigram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,6), max_features = 10000 , encoding='utf8')

for_tf_idf = bigram_vectorizer.fit_transform(df['text'].apply(str.lower).tolist())

tfidf_sparse = TfidfTransformer().fit_transform(for_tf_idf)

tfidf = pd.DataFrame(tfidf_sparse.toarray() , index=df.index, columns=['text_tf_idf_' + str(i) for i in range(tfidf_sparse.shape[1])])


In [5]:
bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,4), max_features = 4000 , encoding='utf8')

for_tf_idf = bigram_vectorizer.fit_transform(df['text'].apply(str.lower).tolist())

tfidf_sparse = TfidfTransformer().fit_transform(for_tf_idf)

tfidf = pd.DataFrame(tfidf_sparse.toarray() , index=df.index, columns=['text_tf_idf_' + str(i) for i in range(tfidf_sparse.shape[1])])


In [6]:
tfidf.shape

(4472, 4000)

In [17]:
from sklearn.preprocessing import MinMaxScaler
saf_df = pd.read_csv('data/saf_feat.csv')
saf_df = pd.DataFrame(MinMaxScaler().fit_transform(saf_df) )

In [18]:
t = pd.concat([tfidf, saf_df],axis=1)

In [21]:

from sklearn.linear_model import LogisticRegression

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'application': 'multiclass',
    'objective':'binary',
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'subsample': 0.8,
    #'colsample_bytree': 0.8,
    #'objective': 'multiclass',
    'nthread': -1,
    'learning_rate': 0.03,
    #'min_child_weight': 50,
    'max_depth': 10
    #'num_class': 2302,
    #'feature_fraction':0.2,
    #'bagging_fraction':0.5
    
}


pred_train = np.zeros(len(df))
scores = []
for itr, ite in tqdm(folds):
    train = t.ix[itr,:]
    val = t.ix[ite,:]
    y_train = y[itr]
    y_val = y[ite]
    #lgb = lgbm.sklearn.LGBMClassifier(n_estimators=1000, seed=0, **params)
    #lgb.fit(train, y_train.values)
    #ypred = lgb.predict_proba(val)[:, 1]
    model = LogisticRegression()
    model.fit(train, y_train.values)
    ypred = model.predict_proba(val)[:, 1]    
    
    
    scores += [accuracy_score(y_val, ypred > 0.5)]
    
scores = np.array(scores)
print(scores.mean(), ' +-', scores.std())

0.6551883589006071  +- 0.012922912910352065


In [ ]:
0.677311041866411  +- 0.020483338513536346

0.6634482861936721  +- 0.01619585664485589


In [20]:
pd.DataFrame(pred_train).to_csv('predictions/tfidf_char_saf.csv', index=False)

In [ ]:
char
0.7260821148929371  +- 0.013283597320227476

word
0.6690396292745285  +- 0.01584611114905873
0.6641249201022691  +- 0.009953519575835854
0.6583008748801534  +- 0.020450857087594386
0.630364133908597  +- 0.007977489608898805
0.6026306327900288  +- 0.009345713266826185


In [20]:
y_train.shape, train.shape

((3576,), (4795, 200))

In [5]:
df['text'] = df['text'].apply(lambda x: str(x))
df_val['text'] = df_val['text'].apply(lambda x: str(x))

In [6]:
df['text'] = df['text'].apply(lambda x: x.lower().split())
df_val['text'] = df_val['text'].apply(lambda x: x.lower().split())

In [17]:
X1 = map(lambda x: ' '.join(x), df['text'])
Y1 = map(lambda x: ' '.join(x), df_val['text'])

hw1 = HashingVectorizer(n_features=1000).fit(df['text'] + df_val['text'])

In [18]:
X1 = pd.DataFrame(hw1.transform(X1).todense())
Y1 = pd.DataFrame(hw1.transform(Y1).todense())

In [19]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X1+Y1)

X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [20]:
y_train = df['coord'].apply(lambda x: len(x) > 0)
y_val = df_val['coord'].apply(lambda x: len(x) > 0)

In [32]:
#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'application': 'multiclass',
    'objective':'binary',
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    'colsample_bytree': 0.8,
    #'objective': 'multiclass',
    'nthread': -1,
    'learning_rate': 0.1,
    #'min_child_weight': 50,
    'max_depth': 9
    #'num_class': 2302,
    #'feature_fraction':0.2,
    #'bagging_fraction':0.5
    
}




#Xdatatrain = xgb.DMatrix(data=X_train.values,
#                             label=y_train.values)
#Xdataval = xgb.DMatrix(data=X_val.values,
#                             label=y_val.values)
#plst = list(param.items())
#watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

#bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=1)

lgb = lgbm.sklearn.LGBMClassifier(n_estimators=700, seed=0, **params)
lgb.fit(X1, y_train.values)
#ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.1, max_depth=9, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=700,
        n_jobs=-1, nthread=-1, num_leaves=31, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=1)

In [33]:
y_pred = lgb.predict_proba(Y1)[:,1]
accuracy_score(y_val, y_pred > 0.5)

0.6467828418230563

In [24]:
0.6812546228946561
0.6794425548455621
0.6595174262734584

In [27]:
y_val.shape

(1492,)

In [30]:
y_val.sum() / len(y_val)

0.5